## Oracle Cloud Instance POC


In [1]:
import os
import json
from pathlib import Path
from typing import Dict

import bravado
from bravado.client import SwaggerClient
# from bravado.requests_client import RequestsClient
from gc3_query.lib.bravado.requests_client import OPCRequestsClient
from bravado.swagger_model import load_file
from secrets import opc_username, opc_password
from bravado_core.exception import MatchingResponseNotFound
from bravado.exception import HTTPBadRequest
from bravado.http_future import HttpFuture

from tinydb import TinyDB, Query


from prettyprinter import pprint, pformat

## https://medium.com/@betz.mark/validate-json-models-with-swagger-and-bravado-5fad6b21a825
# Validate json models with swagger and bravado
from bravado_core.spec import Spec

idm_domain_name = 'gc30003'
idm_service_instance_id = '587626604'
iaas_rest_endpoint = r'https://compute.uscom-central-1.oraclecloud.com'
iaas_auth_endpoint = f'{iaas_rest_endpoint}/authenticate/'

print(f'iaas_rest_endpoint: {iaas_rest_endpoint}')
print(f'iaas_auth_endpoint: {iaas_auth_endpoint}\n')


### Username/pass setup
idm_domain_username = f'/Compute-{idm_domain_name}/{opc_username}'
idm_service_instance_username = f'/Compute-{idm_service_instance_id}/{opc_username}'
# username = traditional_iaas_username
username = idm_service_instance_username
# basic_auth_cred = _basic_auth_str(username, opc_password)


iaas_rest_endpoint: https://compute.uscom-central-1.oraclecloud.com
iaas_auth_endpoint: https://compute.uscom-central-1.oraclecloud.com/authenticate/



In [2]:
json_data = {"user": username, "password": opc_password}
print(f'idm_domain_username: {idm_domain_username}')
print(f'idm_service_instance_username: {idm_service_instance_username}')
print(f'username: {username}')

idm_domain_username: /Compute-gc30003/eric.harris@oracle.com
idm_service_instance_username: /Compute-587626604/eric.harris@oracle.com
username: /Compute-587626604/eric.harris@oracle.com


#### Create DB to store Instance data

In [3]:
instances_db = TinyDB('instances.tdb.json')

####  Authenticate and get header token


In [4]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+directory+json'),
                ])
requests_client = OPCRequestsClient()
requests_client.session.headers.update(headers)

print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
requests_client.session.headers.update(headers)
print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")

response = requests_client.session.post(url=iaas_auth_endpoint, json=json_data)
print(f'Response OK: {response.ok}, Status Code: {response.status_code}, URL: {response.url}')
if response.ok and 'Set-Cookie' in response.headers:
    print(f"Auth request succeess.\n")
    ### The auth cookie is already placed in the session ... nothing else needs to be done.
    print(f"\nSession Cookies: {requests_client.session.cookies}")
    print(f"\nResponse Headers['Set-Cookie']: {response.headers['Set-Cookie']}")
else:
    print(f'Something failed! Response OK: {response.ok}, Status Code: {response.status_code}')

print(f"requests_client.session.headers before update: {requests_client.session.headers}\n")
cookie_header = {'Cookie': response.headers['Set-Cookie']}
print(f"cookie_header: {cookie_header}\n")
requests_client.session.headers.update(cookie_header)
print(f"requests_client.session.headers after update: {requests_client.session.headers}\n")


requests_client.session.headers before update: {'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/oracle-compute-v3+directory+json', 'Connection': 'keep-alive', 'Content-Type': 'application/oracle-compute-v3+json'}

requests_client.session.headers after update: {'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/oracle-compute-v3+directory+json', 'Connection': 'keep-alive', 'Content-Type': 'application/oracle-compute-v3+json'}

Response OK: True, Status Code: 204, URL: https://compute.uscom-central-1.oraclecloud.com/authenticate/
Auth request succeess.


Session Cookies: <RequestsCookieJar[<Cookie nimbula=eyJpZGVudGl0eSI6ICJ7XCJyZWFsbVwiOiBcInVzY29tLWNlbnRyYWwtMVwiLCBcInZhbHVlXCI6IFwie1xcXCJjdXN0b21lclxcXCI6IFxcXCJDb21wdXRlLTU4NzYyNjYwNFxcXCIsIFxcXCJyZWFsbVxcXCI6IFxcXCJ1c2NvbS1jZW50cmFsLTFcXFwiLCBcXFwiZW50aXR5X3R5cGVcXFwiOiBcXFwidXNlclxcXCIsIFxcXCJzZXNzaW9uX2V4cGlyZXNcXFwiOiAxNTIzOTE3OTA2Lj

In [5]:
## Update the swagger spec to use https
spec_file_path = Path().joinpath('open_api_definitions/iaas_instances.json').resolve()
spec_dict = load_file(spec_file_path)
spec_dict['schemes'] = ['https']


In [6]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, application/oracle-compute-v3+directory+json, json, text/html')
                ])

# headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
#                 ('Accept', 'application/oracle-compute-v3+json, json, text/html')])

In [7]:
requests_client = OPCRequestsClient()
requests_client.session.headers.update(headers)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         # config={'also_return_response': True,
                                         #         'validate_responses': True,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         config={'also_return_response': True,
                                                 'validate_responses': False,
                                                 'validate_requests': False,
                                                 'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [8]:
operation_id = "discoverRootInstance"
print(f"Operation {operation_id} starting.")
try:
    operation_future = getattr(instances_resource, operation_id)()
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverRootInstance starting.
REST url for discoverRootInstance: https://compute.uscom-central-1.oraclecloud.com/instance/

discoverRootInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/:
 {'result': ['/Compute-587626604/']}

Operation discoverRootInstance finished.



In [9]:
operation_id = "discoverInstance"
print(f"Operation {operation_id} starting.")
try:
    # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604
    container = operation_details['result'][0].lstrip('/').rstrip('/')
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com
    # container = f"{container}/{opc_username}"
    # # API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/
    container = f"{container}/{opc_username}/"
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.\n")

Operation discoverInstance starting.
container: Compute-587626604/eric.harris@oracle.com/
REST url for discoverInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/

discoverInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/:
 {
    'result': [
        '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/',
        '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-PSFTHR92/',
        '/Compute-587626604/eric.harris@oracle.com/dbaas/',
        '/Compute-587626604/eric.harris@oracle.com/paas/'
    ]
}

Operation discoverInstance finished.



In [10]:
operation_id = "listInstance"
print(f"Operation {operation_id} starting.")
try:
    # container = operation_details['result'][0].lstrip('/').rstrip('/')
    # container = 'Compute-587626604/eric.harris@oracle.com'
    container = 'Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'
    print(f"container: {container}")
    operation_future = getattr(instances_resource, operation_id)(container=container)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    operation_result, operation_response = operation_future.result()
except HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")

Operation listInstance starting.
container: Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1
REST url for listInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/

listInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/:
 {
    'result': [
        '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/'
        'c17bb7ea-724b-4e51-ab72-1bd8714f07b7'
    ]
}

Operation listInstance finished.


In [11]:
headers = dict([('Content-Type', 'application/oracle-compute-v3+json'),
                ('Accept', 'application/oracle-compute-v3+json, json, text/html')])


requests_client = OPCRequestsClient()
requests_client.session.headers.update(headers)
requests_client.session.headers.update(cookie_header)
swagger_client = SwaggerClient.from_spec(spec_dict=spec_dict,
                                         origin_url=iaas_rest_endpoint,
                                         http_client=requests_client,
                                         # config={'also_return_response': True,
                                         #         'validate_responses': True,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         # config={'also_return_response': True,
                                         #         'validate_responses': False,
                                         #         'validate_requests': True,
                                         #         'validate_swagger_spec': True})
                                         config={'also_return_response': True,
                                                 'validate_responses': False,
                                                 'validate_requests': False,
                                                 'validate_swagger_spec': False})
instances_resource = swagger_client.Instances

In [12]:
operation_id = "getInstance"
print(f"\nOperation {operation_id} starting.")
try:
    name = operation_details['result'][0].lstrip('/').rstrip('/')
    print(f"name: {name}")
    operation_future: HttpFuture = getattr(instances_resource, operation_id)(name=name)
    url = operation_future.future.request.url
    print(f"REST url for {operation_id}: {url}")
    print(f"Accept Header=[{operation_future.future.session.headers['Accept']}], Content-Type Header=[{operation_future.future.session.headers['Content-Type']}]")
    operation_result, operation_response = operation_future.result()
except bravado.exception.HTTPBadRequest:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
except bravado.exception.HTTPNotFound:
    print("Request failed for {operation_id}! ")
    print(f"URL: {operation_future.future.request.url}")
    raise
operation_details = json.loads(operation_result)
print("\n{} operation_details:\nHTTP method: {}\nAPI url: {}:\n {}\n".format(operation_id, operation_future.operation.http_method, url, pformat(operation_details)))
print(f"Operation {operation_id} finished.")


Operation getInstance starting.
name: Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
REST url for getInstance: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7
Accept Header=[application/oracle-compute-v3+json, json, text/html], Content-Type Header=[application/oracle-compute-v3+json]

getInstance operation_details:
HTTP method: get
API url: https://compute.uscom-central-1.oraclecloud.com/instance/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/c17bb7ea-724b-4e51-ab72-1bd8714f07b7:
 {
    'domain': 'compute-587626604.oraclecloud.internal.',
    'placement_requirements': [
        '/system/compute/placement/default',
        '/oracle/compute/dedicated/fcbaed9f-242b-42ce-ac77-dd727f9710eb',
        '/system/compute/allow_instances'
    ],
    'ip': '10.19.0.246',
    'fingerprint': '34:a7:bb:00:67:50:85:f4:09:ef:8d:81:90:e7:4

#### Insert results in to DB

In [13]:
instances_db.insert(operation_details)

1

In [15]:
instances_db.all()[-1]

{'account': '/Compute-587626604/default',
 'attributes': {'dns': {'domain': 'compute-587626604.oraclecloud.internal.',
   'hostname': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.',
   'nimbula_vcable-eth0': 'gc3naac-cdmt-lws1.compute-587626604.oraclecloud.internal.'},
  'network': {'nimbula_vcable-eth0': {'address': ['c6:b0:6d:06:65:04',
     '10.19.0.246'],
    'dhcp_options': [],
    'id': '/Compute-587626604/eric.harris@oracle.com/2043e3fa-682b-48a5-8b21-e163a54e7611',
    'model': '',
    'vethernet': '/oracle/public/default',
    'vethernet_id': '0',
    'vethernet_type': 'vlan'}},
  'nimbula_orchestration': '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1',
  'oracle_metadata': {'v1': {'object': '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1/instance',
    'orchestration': '/Compute-587626604/eric.harris@oracle.com/GC3NAAC-CDMT-LWS1'}},
  'sshkeys': ['ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDkl16+TS1DB5jFwxr8tRMfztjLCHK2+wpfqy2eKeUzDJx8oGlasm